# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data1/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.18,79,90,7.47,TF,1714942162
1,1,hermanus,-34.4187,19.2345,12.67,67,24,4.79,ZA,1714942175
2,2,haiku-pauwela,20.9219,-156.3051,25.09,74,40,10.29,US,1714942165
3,3,kenora,49.8167,-94.4337,14.54,47,75,5.66,CA,1714942318
4,4,nyala,12.0500,24.8833,28.71,11,0,3.60,SD,1714942540


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
ideal_weather_df = city_data_df.loc[
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
].dropna()
# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
62,62,nova sintra,14.8667,-24.7167,24.00,79,0,4.11,CV,1714942547
120,120,al muwayh,22.4333,41.7583,24.48,29,0,4.32,SA,1714942555
200,200,ravar,31.2656,56.8055,22.70,32,0,0.56,IR,1714942566
216,216,marsa alam,25.0757,34.8918,25.61,47,0,1.49,EG,1714942568
256,256,badr hunayn,23.7829,38.7905,26.85,38,0,1.29,SA,1714942573
313,313,karratha,-20.7377,116.8463,22.74,43,0,3.82,AU,1714942182
367,367,maceio,-9.6658,-35.7353,26.69,73,0,1.54,BR,1714942448
375,375,tijucas,-27.2414,-48.6336,26.20,96,0,2.15,BR,1714942589
522,522,turabah,21.2141,41.6331,24.97,30,0,3.09,SA,1714942609


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,79,
1,hermanus,ZA,-34.4187,19.2345,67,
2,haiku-pauwela,US,20.9219,-156.3051,74,
3,kenora,CA,49.8167,-94.4337,47,
4,nyala,SD,12.0500,24.8833,11,
...,...,...,...,...,...,...
567,navalmoral de la mata,ES,39.8916,-5.5406,91,
568,vulcan,RO,45.3833,23.2667,92,
569,tabarka,TN,36.9544,8.7580,72,
570,eyl,SO,7.9803,49.8164,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
api_key = geoapify_key
# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{lon},{lat},{radius}"
    bias = f"proximity:{lon},{lat}"
    params = {
        "categories": categories,
        "filter": filters,
        "bias": bias,
        "apiKey": api_key
    }
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df.head()

Starting hotel search
port-aux-francais - nearest hotel: Keravel
hermanus - nearest hotel: Aloe guest house
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
kenora - nearest hotel: Travelodge
nyala - nearest hotel: سوني سفاري
olonkinbyen - nearest hotel: No hotel found
zakynthos - nearest hotel: Phoenix Hotel
port mathurin - nearest hotel: Escale Vacances
vadso - nearest hotel: Scandic Vadsø
tudor vladimirescu - nearest hotel: No hotel found
adrar - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
dudinka - nearest hotel: Виктория
beloyarskiy - nearest hotel: Сибирь
edinburgh of the seven seas - nearest hotel: No hotel found
kingscliff - nearest hotel: Kingscliff Beach Hotel
tazovsky - nearest hotel: Тазовчанка
buka - nearest hotel: No hotel found
paidha - nearest hotel: Jennifer
kapuskasing - nearest hotel: Travelodge
port saint john's - nearest hotel: Outback Inn
balurghat - nearest hotel: No hotel found
vilyuchinsk - nearest hotel: Победа
al kut - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,79,Keravel
1,hermanus,ZA,-34.4187,19.2345,67,Aloe guest house
2,haiku-pauwela,US,20.9219,-156.3051,74,Inn At Mama's Fish House
3,kenora,CA,49.8167,-94.4337,47,Travelodge
4,nyala,SD,12.0500,24.8833,11,سوني سفاري


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoEco",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)